In [1]:
import os
checkpoint_id = "0b6dd99c-b9d9-45d8-978e-1fac4373bbeb"
os.chdir(f"/home/christoph/Code/det-gan-research/checkpoints/{checkpoint_id}/code/")

In [2]:
import cv2
import math
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import scipy.ndimage
import numpy as np
import seaborn as sns
import utils

from ipywidgets import Video
from tqdm.notebook import tqdm
from determined.experimental import Checkpoint
from models import MsgGenerator
from msg_gan import MsgGANTrail
from metrics import InceptionScore

from distributions import TruncatedNormal

toPILImage = transforms.ToPILImage()

In [3]:
trail = Checkpoint.load_from_path("../", map_location=torch.device('cpu'))

TypeError: __init__() missing 1 required positional argument: 'num_classes'

In [ ]:
num_images = 6
truncations=(-1.25, 1.25)
noise_size = trail.latent_dimension
z = utils.sample_noise(num_images, noise_size, truncations=truncations)
dist = utils.sample_noise(1, 100000, truncations=truncations).squeeze()

ax = sns.displot(dist, bins=np.arange(-4, 4, 0.25), kde=True)
plt.show()

In [ ]:
images, w = trail.generator(z)
biggest_size = images[-1].shape[-1]
num_resolutions = int(math.log2(biggest_size))

resolutions, x = trail.generator(z)
fig, axes = plt.subplots(nrows=num_images, ncols=num_resolutions - 1, dpi=200, figsize=(6, 6))

for col, resolution in enumerate(resolutions):
    resolution = utils.shift_image_range(resolution)
    for row, tensor in enumerate(resolution):        
        image = toPILImage(tensor.cpu())
        
        axes[row, col].imshow(image)
        axes[row, col].get_xaxis().set_visible(False)
        axes[row, col].get_yaxis().set_visible(False)

In [ ]:
eval_model, resize_to, num_classes = utils.create_evaluation_model("default")

inception_score = InceptionScore(eval_model, resize_to, num_classes)
z = utils.sample_noise(64, noise_size, truncations=truncations)


images, _ = trail.generator(z)
inception_score.images = images[-1]
inception_score()

In [ ]:
out_dir = "temp"
os.makedirs(out_dir, exist_ok=True)
video_path = 'temp/interpolation.mp4'


time = 30
fps = 30
smoothing = 1.25
total_frames = time * fps

fourcc = cv2.VideoWriter_fourcc(*'VP90')
video = cv2.VideoWriter(video_path, fourcc=fourcc, fps=fps, frameSize=(biggest_size, biggest_size))

all_z = utils.sample_noise(total_frames, noise_size, normalize=False, truncations=truncations)
all_z = scipy.ndimage.gaussian_filter(all_z, [smoothing * fps, 0])
all_z = torch.from_numpy(all_z)

for idx, z in tqdm(enumerate(all_z), desc="Generating video", total=total_frames):
    frame = idx + 1
    z.unsqueeze_(dim=0)
    tensors, w = trail.generator(z)
    tensor = tensors[-1].squeeze()
    tensor = utils.shift_image_range(tensor)
    
    image = toPILImage(tensor.cpu())
    image = np.array(image)
    image = image[:, :, ::-1].copy() 

    video.write(image)
    
video.release()
Video.from_file(video_path, play=True)